In [1]:
import numpy as np
import scipy
from scipy.stats import gamma, beta
from buckley_leverett_monte_carlo import eval_bl_mc

In [2]:
# Main input parameters:
aw = 0.2        # coefficient for perm water - max(krw)
bw = 1.5        # water exponent relative permeability
ao = 0.80       # coefficient for perm oil - max(krw)
bo = 2.0         # oil exponent relative permeability
sw_min = 0.1    # min water saturation values
sw_max = 0.9    # max water saturation values
mu_w = 1.0      # water viscosity in centipoise
mu_o = 4.0      # oil viscosity in centipoise

total_simulation_time = 4.0e5  # s
porosity = 0.3  # m3/m3
diameter = 0.036  # m
length = 0.12  # m
injection_rate = 2e-10  # m3/s
output_times = np.array([2.5e4, 5.0e4, 1.0e5, 2.0e5]) # seconds

number_of_samples = 10000

In [3]:
# Main distributions definitions

In [4]:
# Porosity is defined in an interval between zero and one
# so it makes sense to use a beta distribution

# As there is few, if not only one measurement, the nominal value is known but
# no information about variance is known. Check the effect of change in the variance
# as a function of the nominal value: 5%, 10% and 20% of the nominal value

expected_value = 0.3
factors = [5.0, 10.0, 15.0]
samples = []
pdfs = []
porosity_distributions = []
uloc = np.linspace(0, 1, number_of_samples)

for f in factors:
    std_dev = (f/100.0) * expected_value
    variance = std_dev * std_dev

    a = expected_value*(expected_value * (1.0 - expected_value)/variance - 1.0)
    b = a * (1.0 - expected_value) / expected_value

    distribution = beta(a, b)
    porosity_distributions.append(distribution)
    samples.append(distribution.rvs(size=number_of_samples))
    pdfs.append((uloc, distribution.pdf(uloc)))
    

np.save('porosity_samples', samples)
np.save('porosity_factors', factors)
np.save('porosity_pdfs', pdfs)

In [5]:
# Other properties, such as viscosities, are defined in an interval between zero
# and +infinite so a gamma distribution seems more appropriate (a log-normal could
# also be used)

# Do the same analysis as done for porosity
factors = [5.0, 10.0, 15.0]
expected_value = 1.0
samples = []
pdfs = []
water_viscosity_distributions = []
uloc = np.linspace(0, 2, number_of_samples)

for f in factors:
    std_dev = (f/100.0) * expected_value
    variance = std_dev * std_dev

    theta = variance / expected_value
    k = expected_value * expected_value / variance
    distribution = gamma(k, scale = theta)

    water_viscosity_distributions.append(distribution)

    samples.append(distribution.rvs(size=number_of_samples))
    pdfs.append((uloc, distribution.pdf(uloc)))
    

np.save('water_viscosity_samples', samples)
np.save('water_viscosity_factors', factors)
np.save('water_viscosity_pdfs', pdfs)
# fig1 = plt.figure()
# for s in samples:
#   plt.plot(s, '*')

# fig2 = plt.figure()
# number_of_bins = round(math.sqrt(number_of_samples))
# for f, d, s in zip(factors, water_viscosity_distributions, samples):
#   X_bins, X_freq, X_area , = randvar_pdf(s , number_of_bins)
#   X_binwidth = X_bins[0] - X_bins[1]

#   plt.bar(X_bins , X_freq , X_binwidth , align='edge', edgecolor='k', alpha=0.25)
#   plt.plot(uloc, d.pdf(uloc), '-', label=r'$\sigma$ =' + str(int(f)) + '% of $\mu$')
# plt.legend()
# plt.xlabel('Water viscosity')
# plt.ylabel('Probability density')

In [6]:
expected_value = 4.0
samples = []
pdfs = []
oil_viscosity_distributions = []
uloc = np.linspace(2, 6, number_of_samples)
factors = [2.5, 5.0, 7.5]
for f in factors:
    std_dev = (f/100.0) * expected_value
    variance = std_dev * std_dev

    theta = variance / expected_value
    k = expected_value * expected_value / variance
    distribution = gamma(k, scale = theta)

    oil_viscosity_distributions.append(distribution)
    samples.append(distribution.rvs(size=number_of_samples))
    pdfs.append((uloc, distribution.pdf(uloc)))
    

np.save('oil_viscosity_samples', samples)
np.save('oil_viscosity_factors', factors)
np.save('oil_viscosity_pdfs', pdfs)
#   samples.append(distribution.rvs(size=number_of_samples))

# fig1 = plt.figure()
# for s in samples:
#   plt.plot(s, '*')

# fig2 = plt.figure()
# for f, d, s in zip(factors, oil_viscosity_distributions, samples):
#   X_bins, X_freq, X_area , = randvar_pdf(s , number_of_bins)
#   X_binwidth = X_bins[0] - X_bins[1]

#   plt.bar(X_bins , X_freq , X_binwidth , align='edge', edgecolor='k', alpha=0.25)
#   plt.plot(uloc, d.pdf(uloc), '-', label=r'$\sigma$ =' + str(f) + '% of $\mu$')
# plt.legend()
# plt.xlabel('Oil viscosity')
# plt.ylabel('Probability density')

In [7]:
xds = []
means_sw = []
sws_upp = []
sws_low = []
times = []
means_npd = []
npds_upp = []
npds_low = []
npds_squared_integral = []

for poro_dist, wat_visc_dist, oil_visc_dist, in zip(porosity_distributions, water_viscosity_distributions, oil_viscosity_distributions):

    # Here 10,000 samples are being used as from the plot above the smallest sample size that should be used
    # is in the order of 10^4 as the second moment does not change significantly anymore
    xd, mean_sw, sw_upp, sw_low, time, mean_npd, npd_upp, npd_low, npd_squared_integral = eval_bl_mc(
        number_of_samples,
        poro_dist, wat_visc_dist,
        oil_visc_dist,
        total_simulation_time, 
        diameter, 
        length, 
        injection_rate, 
        aw, 
        bw, 
        ao, 
        bo, 
        sw_min, 
        sw_max, 
        output_times,
    )
    xds.append(xd)
    means_sw.append(mean_sw)
    sws_upp.append(sw_upp)
    sws_low.append(sw_low)
    times.append(time)
    means_npd.append(mean_npd)
    npds_upp.append(npd_upp)
    npds_low.append(npd_low)
    npds_squared_integral.append(npd_squared_integral)

xds = np.array(xds)
means_sw = np.array(means_sw)
sws_upp = np.array(sws_upp)
sws_low = np.array(sws_low)
times = np.array(times)
means_npd = np.array(means_npd)
npds_upp = np.array(npds_upp)
npds_low = np.array(npds_low)
npds_squared_integral = np.array(npds_squared_integral)
    
np.save('mc_xds', xds)
np.save('mc_means_sw', means_sw)
np.save('mc_sws_upp', sws_upp)
np.save('mc_sws_low', sws_low)
np.save('mc_times', times)
np.save('mc_means_npd', means_npd)
np.save('mc_npds_upp', npds_upp)
np.save('mc_npds_low', npds_low)
np.save('mc_npds_squared_integral', npds_squared_integral)
np.save('mc_output_times', output_times)

C:\ProgramData\Miniconda3\envs\buckley\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\dev\uq-buckley-leverett\buckley_leverett_monte_carlo.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  evals = np.array([ model(sample[0], sample[1] , sample[2]) for sample in samples.T ])
C:\ProgramData\Miniconda3\envs\buckley\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
C:\dev\uq-buckley-leverett\buckley_leverett.py:17: RuntimeWarning: invalid value encountered in power
  kw = krw0 * np.power(s_hat, nw)
